In [ ]:
!pip install pinecone
!pip install transformers
!pip install sentence-transformers

In [ ]:
import pandas as pd
data = pd.read_csv('fincancial_literacy_resources.csv')

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

embeddings = model.encode(desc)

embeddings

In [ ]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key='API Key')

index_name = "fidhacks"
dimension = 768  # Dimension of DistilBERT embeddings

if index_name not in pc.list_indexes().names():
  pc.create_index(
      name=index_name,
      dimension=dimension,
      metric="cosine",
      spec=ServerlessSpec(
          cloud='aws',
          region='us-east-1'
      )
  )

In [ ]:
vectors_with_metadata = [
    {
        "id": str(i),
        "values": embedding,
        "metadata": {
            "title": row["Title"],
            "description": row["Description"],
            "link": row["Link"]
        }
    }
    for i, (embedding, row) in enumerate(zip(embeddings, data.to_dict(orient="records")))
]

In [ ]:
index = pc.Index(index_name)
index.upsert(vectors=vectors_with_metadata, namespace="auto_loan_resources")

In [ ]:
test_vect = model.encode("""You can take a first step toward learning more
about your credit history by checking your credit
scores and credit reports. The good news is you are
increasingly able to see your credit scores for free.
To raise awareness of how you can access and
use your scores, the CFPB is releasing a list of
companies that offer their credit card customers
free access to one of their credit scores.""")
test_vect[0]

In [ ]:
# Query the Pinecone index
query_results = index.query(
    namespace="auto_loan_resources",
    vector=test_vect.tolist(),
    top_k=1,
    include_values=True,
    include_metadata=True
)

# Print the query results
print(query_results)

if query_results.matches:
    top_match_metadata = query_results.matches[0].metadata
    print("Top Match Metadata:")
    print("Title:", top_match_metadata.get("title"))
    print("Description:", top_match_metadata.get("description"))
    print("Link:", top_match_metadata.get("link"))
else:
    print("No matches found.")